| File            | Mục đích                                                    | Cột chính                                  |
| --------------- | ----------------------------------------------------------- | ------------------------------------------ |
| **movies.csv**  | Thông tin phim (metadata)                                   | `movieId`, `title`, `genres`               |
| **ratings.csv** | Dữ liệu đánh giá (interaction chính)                        | `userId`, `movieId`, `rating`, `timestamp` |
| **tags.csv**    | Ghi chú / từ khóa do người dùng gắn cho phim (metadata phụ) | `userId`, `movieId`, `tag`, `timestamp`    |


⚙️ Bước 1: Đọc dữ liệu

In [1]:
import pandas as pd

movies = pd.read_csv(r'dataset\movies.csv')
ratings = pd.read_csv(r'dataset\ratings.csv')
tags = pd.read_csv(r'dataset\tags.csv')

print(movies.head())
print(ratings.head())
print(tags.head())
print(len(movies), len(ratings), len(tags))


   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931
   userId  movieId              tag   timestamp
0       2    60756            funny  1445714994
1       2    60756  Highly quotable  1445714996
2       2    60756     will ferre

Làm sạch dữ liệu cơ bản

In [2]:
print(movies.isnull().sum())
print(ratings.isnull().sum())
print(tags.isnull().sum())
movies = movies.dropna()
ratings = ratings.dropna()
tags = tags.dropna(subset=['tag'])

movieId    0
title      0
genres     0
dtype: int64
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64
userId       0
movieId      0
tag          0
timestamp    0
dtype: int64


In [3]:
ratings = ratings.drop_duplicates()
tags = tags.drop_duplicates()
print(len(movies), len(ratings), len(tags))


9742 100836 3683


Gộp dữ liệu

In [4]:
data = pd.merge(ratings, movies, on='movieId', how='left')
print(data.head())

   userId  movieId  rating  timestamp                        title  \
0       1        1     4.0  964982703             Toy Story (1995)   
1       1        3     4.0  964981247      Grumpier Old Men (1995)   
2       1        6     4.0  964982224                  Heat (1995)   
3       1       47     5.0  964983815  Seven (a.k.a. Se7en) (1995)   
4       1       50     5.0  964982931   Usual Suspects, The (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                               Comedy|Romance  
2                        Action|Crime|Thriller  
3                             Mystery|Thriller  
4                       Crime|Mystery|Thriller  


In [5]:
# Làm sạch cột 'genres'
movies['genres'] = movies['genres'].str.replace('|', ',', regex=False)

print("----- Dữ liệu cột 'genres' sau khi xử lý -----")

# Gộp dữ liệu
data = pd.merge(ratings, movies, on='movieId', how='left')

# Bỏ cột 'timestamp'
data = data.drop(columns=['timestamp'])

# Lưu file
data.to_csv('merged_movie_data.csv', index=False, encoding='utf-8')
print("đã lưu 👍:")
print(data.head())


----- Dữ liệu cột 'genres' sau khi xử lý -----
đã lưu 👍:
   userId  movieId  rating                        title  \
0       1        1     4.0             Toy Story (1995)   
1       1        3     4.0      Grumpier Old Men (1995)   
2       1        6     4.0                  Heat (1995)   
3       1       47     5.0  Seven (a.k.a. Se7en) (1995)   
4       1       50     5.0   Usual Suspects, The (1995)   

                                        genres  
0  Adventure,Animation,Children,Comedy,Fantasy  
1                               Comedy,Romance  
2                        Action,Crime,Thriller  
3                             Mystery,Thriller  
4                       Crime,Mystery,Thriller  


In [6]:
data = pd.read_csv('merged_movie_data.csv')
print(data.head())

   userId  movieId  rating                        title  \
0       1        1     4.0             Toy Story (1995)   
1       1        3     4.0      Grumpier Old Men (1995)   
2       1        6     4.0                  Heat (1995)   
3       1       47     5.0  Seven (a.k.a. Se7en) (1995)   
4       1       50     5.0   Usual Suspects, The (1995)   

                                        genres  
0  Adventure,Animation,Children,Comedy,Fantasy  
1                               Comedy,Romance  
2                        Action,Crime,Thriller  
3                             Mystery,Thriller  
4                       Crime,Mystery,Thriller  


import thư viện cần thiết

```python

In [48]:
# Cell 1
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split

# Đọc dữ liệu
data = pd.read_csv('merged_movie_data.csv')
# data = data.sample(10000, random_state=42)
print("📄 Dữ liệu mẫu:")
print(data.head())


📄 Dữ liệu mẫu:
   userId  movieId  rating                        title  \
0       1        1     4.0             Toy Story (1995)   
1       1        3     4.0      Grumpier Old Men (1995)   
2       1        6     4.0                  Heat (1995)   
3       1       47     5.0  Seven (a.k.a. Se7en) (1995)   
4       1       50     5.0   Usual Suspects, The (1995)   

                                        genres  
0  Adventure,Animation,Children,Comedy,Fantasy  
1                               Comedy,Romance  
2                        Action,Crime,Thriller  
3                             Mystery,Thriller  
4                       Crime,Mystery,Thriller  


In [50]:
# Loại bỏ phim trùng lặp trước khi xử lý genres
data = data.drop_duplicates(subset=['movieId', 'title']).reset_index(drop=True)

# Kiểm tra số lượng sau khi loại trùng
print("✅ Số phim duy nhất còn lại:", len(data))
print(data[['movieId', 'title']].head())

✅ Số phim duy nhất còn lại: 9724
   movieId                        title
0        1             Toy Story (1995)
1        3      Grumpier Old Men (1995)
2        6                  Heat (1995)
3       47  Seven (a.k.a. Se7en) (1995)
4       50   Usual Suspects, The (1995)


In [51]:
# Cell 2
# Điền giá trị trống trong 'genres'
data['genres'] = data['genres'].fillna('')

# Chuẩn hóa chuỗi: bỏ khoảng trắng thừa
data['genres'] = data['genres'].str.replace(' ', '')

# Kiểm tra lại dữ liệu sau xử lý
print("✅ Sau xử lý cột genres:")
print(data[['title', 'genres']].head())


✅ Sau xử lý cột genres:
                         title                                       genres
0             Toy Story (1995)  Adventure,Animation,Children,Comedy,Fantasy
1      Grumpier Old Men (1995)                               Comedy,Romance
2                  Heat (1995)                        Action,Crime,Thriller
3  Seven (a.k.a. Se7en) (1995)                             Mystery,Thriller
4   Usual Suspects, The (1995)                       Crime,Mystery,Thriller


In [52]:
# Cell 3 — xử lý lại TF-IDF an toàn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Reset lại index để đồng bộ thứ tự
data = data.reset_index(drop=True)

# TF-IDF vectorizer
tfidf = TfidfVectorizer(token_pattern=r"[^,]+")
tfidf_matrix = tfidf.fit_transform(data['genres'])

# Tính cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Tạo mapping chính xác: title → vị trí (0..len(data)-1)
indices = pd.Series(data.index, index=data['title']).drop_duplicates()

print("✅ TF-IDF và cosine similarity đã tạo đồng bộ!")


✅ TF-IDF và cosine similarity đã tạo đồng bộ!


In [53]:
# Cell 4
# Chuẩn bị dữ liệu cho Surprise
reader = Reader(rating_scale=(0.5, 5.0))
dataset = Dataset.load_from_df(data[['userId', 'movieId', 'rating']], reader)

# Chia tập train/test
trainset, testset = train_test_split(dataset, test_size=0.3, random_state=42)

# Khởi tạo mô hình SVD
model = SVD(n_factors=50, random_state=42)
model.fit(trainset)

print("✅ Huấn luyện mô hình SVD xong!")


✅ Huấn luyện mô hình SVD xong!


In [54]:
def hybrid_recommend(user_id, title, top_n=5):
    if title not in indices:
        return f"❌ Phim '{title}' không tồn tại trong dataset."
    
    idx = indices[title]
    if isinstance(idx, pd.Series):  # nếu có nhiều bản trùng
        idx = idx.iloc[0]           # chọn bản đầu tiên

    # --- Content-based ---
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:50]
    
    movie_indices = [i[0] for i in sim_scores]
    similar_movies = data.iloc[movie_indices][['movieId', 'title', 'genres']].copy()

    # --- Collaborative Filtering ---
    similar_movies['est_rating'] = similar_movies['movieId'].apply(
        lambda x: model.predict(user_id, x).est
    )

    # --- Trả kết quả ---
    recommendations = similar_movies.sort_values('est_rating', ascending=False).head(top_n)
    return recommendations[['title', 'genres', 'est_rating']]


In [55]:
movie_input = "Toy Story (1995)"
print(indices[movie_input])
print(cosine_sim.shape)


0
(9724, 9724)


In [56]:
user_id = 1
movie_input = "Toy Story (1995)"

print(f"🎬 Gợi ý cho user {user_id} dựa trên phim '{movie_input}':\n")
recommendations = hybrid_recommend(user_id, movie_input, top_n=5)
print(recommendations)


🎬 Gợi ý cho user 1 dựa trên phim 'Toy Story (1995)':

                                    title  \
135         Lord of the Rings, The (1978)   
744                          Shrek (2001)   
4439  Last Year's Snow Was Falling (1983)   
4348                        Presto (2008)   
751                 Monsters, Inc. (2001)   

                                                 genres  est_rating  
135                Adventure,Animation,Children,Fantasy    4.556899  
744   Adventure,Animation,Children,Comedy,Fantasy,Ro...    4.544878  
4439                  Animation,Children,Comedy,Fantasy    4.539099  
4348                  Animation,Children,Comedy,Fantasy    4.525394  
751         Adventure,Animation,Children,Comedy,Fantasy    4.521306  
